# Clase práctica (90 min) — Limpieza y análisis en **Google Sheets** con dataset de Taylor Swift

**Clase teórica (sin ejecución de código).**  
**Fecha:** 2025-08-18
**Objetivo general:** Aprender a **limpiar datos heterogéneos** y construir indicadores en *Google Sheets* usando un flujo reproducible y auditable (con `change_log`).  
**Dataset:** `taylor_swift_da_intro.xlsx` (hojas: `raw_songs`, `clean_songs`, `lookup_albums`, `change_log`).

<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/w2s1.jpg" width="500">
</div>

---

## Introducción
En esta sesión aprenderás a **detectar y corregir problemas comunes de calidad de datos** y a **transformar columnas** para preparar la información antes del análisis. Luego, convertirás los datos limpios en **métricas y visualizaciones** claras para comunicar hallazgos.

Usaremos el archivo de Taylor Swift como hilo conductor. La práctica operativa será en Excel/Google Sheets (no dentro de este notebook).


## Agenda (90 min)
1. (10’) Setup y reglas (no tocar `raw_*`, registrar cambios en `change_log`).
2. (25’) Normalización básica: álbum, fechas, booleanos y numéricos.
3. (20’) Duración a segundos y *split* de autores (transformaciones de texto).
4. (15’) Duplicados y llaves de unicidad.
5. (10’) KPIs y resumen (con y sin tabla dinámica).
6. (10’) Gráficos rápidos y checklist final.
7. (≈5’) **Resumen de aprendizaje y mini evaluación**.

---

## 0) Setup inicial

**Paso 1 — Subir archivo a Google Sheets**
- Descarga `taylor_swift_da_intro.xlsx` y súbelo a **Google Drive**. Ábrelo con **Google Sheets**.

**Paso 2 — Hoja de trabajo**
- Duplica `raw_songs` → nómbrala `clean_songs`. **Nunca** edites `raw_songs`.
- Congela la fila 1, ajusta anchos, activa filtro.

**Paso 3 — Bitácora de cambios**
- Abre `change_log` y agrega tu primera fila:
  - `timestamp`: `=NOW()`  
  - `user`: tu nombre  
  - `sheet`: `"clean_songs"`  
  - `action`: `"create"`  
  - `details`: `"Duplicada desde raw_songs"`  
- Registra **toda modificación estructural** (columnas nuevas, deduplicación, decisiones).

**Buenas prácticas**: Usa colores suaves para nuevas columnas, y comentarios en encabezados para documentar (Insertar → Nota).

---
## 1) Normalización básica (álbum, fechas, booleanos, numéricos)

### 1.1 Normalizar nombre de álbum (`album_norm`)
**Objetivo:** Unificar variantes como `FEARLESS`, `Speak now `, `EverMore`, `Mid nights`, etc.

**Inserta columna:** `album_norm` (junto a `album`).  
**Opción A — Reglas con patrones (sin tabla lookup):**

```gs
=SWITCH(B2,
 "Taylor Swift","Taylor Swift",
 "Fearless","Fearless",
 "Speak Now","Speak Now",
 "Red","Red",
 "1989","1989",
 "Reputation","Reputation",
 "Lover","Lover",
 "Folklore","Folklore",
 "Evermore","Evermore",
 "Midnights","Midnights",
 B2
)
```
**Explicación clave:**
- `TRIM` elimina espacios extra; `LOWER` estandariza a minúsculas para comparar.  
- `REGEXMATCH` permite **tolerar** errores típicos (p.ej., `Mid nights` → `Midnights`).  
- `IFS` aplica la **primera** condición verdadera (orden importa).  
- `PROPER` (última rama) corrige capitalización cuando no se cumple ninguna regla anterior.

**Opción B — Con tabla de referencia (`lookup_albums`)**  
Más mantenible cuando la lista crece:
```gs
=IFERROR(
  VLOOKUP(PROPER(TRIM(B2)), lookup_albums!A:B, 2, FALSE),
  PROPER(TRIM(B2))
)
```
- `VLOOKUP` busca la versión normalizada en una tabla controlada por el equipo.  
- `IFERROR` evita errores y retorna una versión “digna” (`PROPER(TRIM)`) si no encuentra coincidencia exacta.

---

### 1.2 Fechas a formato ISO `yyyy-mm-dd` (`release_iso`)
**Problema:** Fechas mezcladas (`2008-11-11`, `11/11/2008`, `11-11-2008`, etc.).  
**Objetivo:** Homologar a ISO para ordenar, filtrar y agrupar sin ambigüedad.

```gs
=TEXT(DATEVALUE(SUBSTITUTE(D2, ".", "/")), "yyyy-mm-dd")
=TEXT(DATEVALUE(SUBSTITUTE(D2, "-", "/")), "yyyy-mm-dd")
=TEXT(D2, "yyyy-mm-dd")
```
**Explicación clave:**
- `SUBSTITUTE` estandariza separadores a `-` y compacta espacios.  
- `DATEVALUE` **interpreta** la fecha; si falla, intentamos una segunda lectura con el valor crudo.  
- `TEXT(...,"yyyy-mm-dd")` **impone** el formato ISO.  
- `IFERROR` captura entradas inválidas y retorna vacío `""` para auditarlas luego.

> *Tip:* Si trabajas con configuraciones regionales diferentes, valida que el separador y orden día/mes no cambien la interpretación. Si dudas, aplica `DATE(year,month,day)` con `REGEXEXTRACT` explícito.

---

### 1.3 Booleanos como TRUE/FALSE (`explicit_bool`, `is_single`)
**Problema:** Valores como `Yes`, `no`, `TRUE`, `0/1`.  
**Solución:** Convertir a `TRUE/FALSE` con un mapeo robusto.

```gs
=OR(LOWER(J2)="yes", LOWER(J2)="true", J2=1)

```
**Explicación clave:**
- Forzamos a minúsculas con `LOWER` y evaluamos con una sola expresión regular.  
- Estandarizar booleanos evita errores en filtros, `COUNTIF`, y tablas dinámicas.

---

### 1.4 Números limpios (`streams_num`, `chart_peak_num`)
**Problema:** `streams` viene con comas, guiones bajos o espacios (`"1,800,000,000"`, `"1_000_000_000"`, `"1 200 000 000"`).  
**Solución — limpiar y convertir:**

```gs
=VALUE(SUBSTITUTE(SUBSTITUTE(SUBSTITUTE(G2,",",""),"_","")," ",""))
```
**Explicación:** Encadenamos `SUBSTITUTE` para remover distintos separadores, y `VALUE` convierte texto → número.

**`chart_peak`** puede traer vacíos o texto:
```gs
=IFERROR(VALUE(K2),)
```
Devuelve número o `blank` si no es convertible.

> **Registra** en `change_log` que agregaste columnas de normalización; describe brevemente las reglas aplicadas.


---
## 2) Duración uniforme y *split* de autores

### 2.1 Duración a segundos (`duration_sec`)
**Problema:** Mezcla de formatos: `mm:ss`, `3m51s`, o solo segundos (`235`).  
**Meta:** Llevar todo a **segundos** para comparaciones y promedios.

```gs
=IF(
  ISNUMBER(SEARCH(":", SUBSTITUTE(SUBSTITUTE(LOWER(TRIM(N2)),"m",":"),"s",""))),
  VALUE(LEFT(SUBSTITUTE(SUBSTITUTE(LOWER(TRIM(N2)),"m",":"),"s",""),
             FIND(":",SUBSTITUTE(SUBSTITUTE(LOWER(TRIM(N2)),"m",":"),"s",""))-1))*60
  + VALUE(MID(SUBSTITUTE(SUBSTITUTE(LOWER(TRIM(N2)),"m",":"),"s",""),
              FIND(":",SUBSTITUTE(SUBSTITUTE(LOWER(TRIM(N2)),"m",":"),"s",""))+1,99)),
  VALUE(TRIM(N2))
)

```
**Cómo funciona:**
- Caso `mm:ss`: extraemos minutos y segundos por separado y calculamos `min*60 + seg`.  
- Caso `XmYYs`: extraemos `m` y `s` (siempre 2 dígitos de segundos) y convertimos.  
- Caso numérico: `VALUE` asume segundos ya limpios.  
- Si algo no calza, revisa celdas con `#ERROR` y corrige fuente.

---

### 2.2 Separar y limpiar **writers** en la misma hoja (sin Apps Script)

**Objetivo:** Obtener los autores de `writers` en columnas separadas y en formato limpio, **sin crear pestañas nuevas**.

---

#### Paso 1 — Inserta columnas para autores
A la derecha de `writers`, inserta 3 columnas con encabezados: `writer_1`, `writer_2`, `writer_3` (puedes añadir más si hace falta).

#### Paso 2 — Divide por coma con `SPLIT`
En `writer_1` de la fila 2 escribe y copia hacia abajo:
```gs
=SPLIT(TRIM(H2), ",")
```
**Qué hace:** Divide el contenido de `H2` por comas y reparte cada autor en columnas contiguas. `TRIM` limpia espacios al inicio/fin.

*Ejemplo:*  
`Taylor Swift, Max Martin, Shellback` → `Taylor Swift | Max Martin | Shellback`

#### Paso 3 — Limpia nombres (espacios y capitalización)
Crea columnas `writer_1_clean`, `writer_2_clean`, `writer_3_clean` y en cada una coloca:
```gs
=IF(M2="","",PROPER(REGEXREPLACE(M2," +"," ")))
```
*Usa la columna correspondiente (por ejemplo, si `writer_1` está en la columna M, usa `M2` en lugar de `A2`).*

**Qué hace:**
- `REGEXREPLACE(" +"," ")` comprime espacios múltiples en uno.
- `PROPER` convierte a “Nombre Propio” (Taylor Swift).
- Copia/arrastra la fórmula hacia abajo para aplicarla a todas las filas.

#### Paso 4 — (Opcional) Lista unificada limpia
Si prefieres una sola columna con los autores normalizados, crea `writers_clean_joined`:
```gs
=IF(H2="","",TRIM(PROPER(SUBSTITUTE(H2, ",", ", "))))
```
**Qué hace:** Estandariza mayúsculas/minúsculas y asegura espacios después de cada coma.

#### Paso 5 — (Opcional) Conteo por autor (en la misma hoja o en una nueva)
Para obtener un ranking de autorías sin crear muchos pasos intermedios:
```gs
=QUERY(SPLIT(FLATTEN(writers_clean_joined), ","),
 "select TRIM(Col1), count(Col1)
  where Col1 is not null
  group by TRIM(Col1)
  order by count(Col1) desc", 1)
```
*(Si `writers_clean_joined` tiene otro nombre o está en otra columna, reemplázalo en la fórmula.)*

---

**Notas didácticas clave**
- Usa **SPLIT** cuando quieras “repartir” texto en varias columnas.
- **TRIM** y **REGEXREPLACE** limpian espacios problemáticos.
- **PROPER** estandariza nombres propios.
- El conteo final usa `FLATTEN + SPLIT + QUERY`; es opcional y se puede colocar al final de la misma hoja.


---
## 3) (Opcional) Duplicados y llaves de unicidad

### 3.1 Clave canónica (`key`)
**Objetivo:** Detectar duplicados reales (variantes de `song` + `album` + `track_no`).  
En `clean_songs`, crea una columna `key` que concatene **versión normalizada** de `song`, `album_norm` y `track_no` con formato fijo:

```gs
=LOWER(REGEXREPLACE(TRIM(A2)," +"," ")) & "||" & LOWER(TRIM(L2)) & "||" & TEXT(C2,"00")
```
**Explicación:**
- `REGEXREPLACE(...," +"," ")` comprime espacios múltiples.  
- `LOWER` + `TRIM` garantizan comparaciones consistentes.  
- `TEXT(C2,"00")` fuerza dos dígitos para el número de pista (`01`, `09`, `10`).  
- Separador `||` reduce colisiones accidentales.

### 3.2 Bandera de duplicado (`dupe_flag`)
```gs
=COUNTIF($Q$2:$Q,$Q2)>1
```
- Cuenta cuántas veces aparece la clave. Si es > 1, **hay duplicados**.

### 3.3 Resolver duplicados por regla de negocio
**Ejemplo de criterio:** conservar la versión con mayor `streams_num` (calidad/éxito).  
Primero, crea un rango **filtrado** que excluya duplicados indeseados y ordena:

```gs
=INDEX(SORT(FILTER(A2:Z, NOT(R2:R)), H2:H, FALSE))
```
- `FILTER(..., NOT(R2:R))` descarta filas marcadas como duplicadas “perdedoras”.  
- `SORT(..., H2:H, FALSE)` ordena por streams descendente para auditoría.  
> Ajusta columnas del `FILTER`/`SORT` según tu estructura real. Documenta en `change_log` **qué fila se conservó y por qué**.


---
## 4) KPIs y Resumen (con y sin Tabla Dinámica)

### 4.1 Sin Pivot — Totales y promedios por álbum
En una pestaña `summary_albums`:

**Lista de álbumes únicos (A2):**
```gs
=SORT(UNIQUE(clean_songs!L2:L))
```
**Suma de streams por álbum (B2):**
```gs
=IF($A2="","",SUMIF(clean_songs!L:L, $A2, clean_songs!H:H))
```
**Promedio de duración (seg) por álbum (C2):**
```gs
=IF($A2="","",AVERAGEIF(clean_songs!L:L, $A2, clean_songs!O:O))
```
**Explicación:**
- `UNIQUE` crea el universo de dimensiones.  
- `SUMIF` y `AVERAGEIF` **no** requieren tablas dinámicas y son fáciles de auditar.  
- Copia (arrastra) las fórmulas hacia abajo para cubrir todos los álbumes listados.

**KPI Streams por minuto (D2):**
```gs
=IF($C2="","", $B2 / ($C2/60))
```
- Convierte duración promedio en **minutos** (`C/60`) y calcula la tasa.

---

### 4.2 Con Tabla Dinámica (Pivot)
**Insertar → Tabla dinámica** usando `clean_songs`:
- **Filas:** `album_norm`  
- **Valores:** `streams_num` (Suma), `duration_sec` (Promedio)  
- **Opcional:** `explicit_bool` (Recuento de verdaderos)  
- Ordena por `streams_num` descendente.  
**Ventaja:** Consolidación rápida; **Desventaja:** fórmulas menos visibles que en celdas.

---

### 4.3 Top N canciones por streams (sin Pivot)
Pestaña `top_songs`:
```gs
=QUERY(clean_songs!A:Z,
 "select A, L, H
  order by H desc
  limit 10", 1)
```
- `QUERY` permite ordenar/limitar con sintaxis tipo SQL.


---
## 5) Gráficos y checklist final

**Gráficos en Sheets:**
1. Columnas: `streams_sum` por `album_norm` (desde `summary_albums`).  
2. Histograma: distribución de `duration_sec` (en `clean_songs`).  
3. Formatos:  
   - `streams` → formato número con separador de miles.  
   - `dur_avg/60` → número con 1–2 decimales (minutos).

**Checklist final:**
- `change_log` completo y claro (qué, dónde, por qué).  
- `clean_songs` sin duplicados y con tipos correctos (fechas ISO, booleanos, numéricos).  
- `summary_albums` y/o Pivot coherentes con los datos.  
- Archivo compartido con el instructor y con comentarios en encabezados donde aplique.


---
## 6) Resumen de aprendizaje (takeaways)
- Normalizar **texto** con `TRIM`, `LOWER/UPPER/PROPER`, `REGEXREPLACE`, `IFS`.  
- Homologar **fechas** con `SUBSTITUTE` + `DATEVALUE` + `TEXT` en ISO.  
- Convertir **booleanos** con `REGEXMATCH` para múltiples variantes.  
- Limpiar **números** heterogéneos con `SUBSTITUTE` + `VALUE`.  
- Transformar **duración** a segundos con `REGEXEXTRACT` y condiciones.  
- Pasar de **wide → long** con `FLATTEN` + `SPLIT` + `QUERY`.  
- Detectar **duplicados** con claves canónicas y reglas de negocio.  
- Construir **KPIs** sin Pivot (SUMIF/AVERAGEIF) o con Pivot según convenga.

---

## 7) Mini evaluación (5 ítems)

**1) Verdadero/Falso:**  
> `TRIM` elimina **todos** los espacios dentro del texto, no solo los iniciales/finales.  
**Respuesta esperada:** *Falso*. `TRIM` quita espacios al inicio/fin y reduce múltiples a uno, pero **no** elimina *todos* los espacios internos.

**2) Verdadero/Falso:**  
> `IFERROR` es útil para diseñar flujos de limpieza porque evita que una celda con error rompa cálculos encadenados.  
**Respuesta esperada:** *Verdadero*.

**3) Opción múltiple:** ¿Cuál combinación estandariza mejor un campo textual irregular para comparaciones?
- A) `LOWER(A2)`  
- B) `TRIM(A2)`  
- C) `PROPER(A2)`  
- D) `LOWER(TRIM(REGEXREPLACE(A2," +"," ")))`  
**Respuesta esperada:** *D* (aplica compresión de espacios + trim + lower).

**4) Práctica breve:**  
En `clean_songs`, crea una columna `release_iso` que convierta fechas como `11/10/2017`, `2010-10-25`, `11-11-2008` a `yyyy-mm-dd`.  
**Pista de fórmula:** Usa `SUBSTITUTE` para unificar separadores + `DATEVALUE` + `TEXT`, protegido con `IFERROR`.

**5) Práctica breve:**  
En `summary_albums`, calcula `streams_per_min` usando suma de `streams_num` y promedio de `duration_sec`.  
**Pista de fórmula:**
```gs
=IF($C2="","", $B2 / ($C2/60))
```
**Rúbrica rápida (auto-chequeo):**  
- (1 pt) Normalizaste álbumes con reglas o lookup.  
- (1 pt) Fechas ISO correctas.  
- (1 pt) Booleanos y numéricos consistentes.  
- (1 pt) Duración a segundos y *split* de autores funcional.  
- (1 pt) KPIs listos y explicados.


---
## Rúbrica sugerida (evaluación formativa)
- Detección/corrección de calidad (duplicados, faltantes, consistencia) — **30%**  
- Transformaciones de columna (texto/fecha/número/listas) — **25%**  
- Flujo claro y `change_log` completo — **15%**  
- Estadísticos resumen correctos y tratamiento de nulos — **15%**  
- Gráfico claro + narrativa — **10%**  
- Uso responsable de IA documentado — **5%**

## Cierre
- Primero **limpiar**, luego **analizar** y finalmente **comunicar**.  
- Las reglas explícitas y la trazabilidad inspiran confianza.  
- Un gráfico simple y una narrativa concisa son más efectivos que complejos sin contexto.

## Siguientes Pasos
- **Próxima sesión:** profundizaremos en el ecosistema de datos y en el flujo de punta a punta.
- **Participación continua:** asistir a Co-Learning y a Sprint Focus, y usar los canales de Discord para hacer preguntas.
- **Recordatorios:** la grabación y recursos utilizados, se comparten al finalizar la sesión; en caso de necesitar apoyo adicional, agenda un 1:1.

## Siguientes Pasos
- **Próxima sesión:** Sprint 2 - Formulación de preguntas de negocio.
- **Participación:** Intenta limpiar el dataset de práctica adicional por tu cuenta.
- **Recordatorios:** Sube tu ejercicio práctico si es requerido.
